In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [12]:
pd.set_option('max_columns', None)

In [2]:
data = pd.read_csv('Housing_Prices/train.csv')

data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# Create X and y
y = data[['SalePrice']]
X = data.drop(columns=['SalePrice', 'Id'], axis=1)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [4]:
# Save y
y_train.to_csv('data/y_train.csv', index=False)
y_val.to_csv('data/y_val.csv', index=False)
y_test.to_csv('data/y_test.csv', index=False)

In [5]:
# Remove "object"-type features from X
cont_features = [col for col in X.columns if X[col].dtype in [np.float64, np.int64]]

# Remove "object"-type features from X_train and X_test
X_train_cont = X_train.loc[:, cont_features]
X_val_cont = X_val.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Impute missing values with median using SimpleImputer
impute = SimpleImputer(strategy='median')
X_train_numeric = pd.DataFrame(impute.fit_transform(X_train_cont), columns=cont_features)
X_val_numeric = pd.DataFrame(impute.transform(X_val_cont), columns=cont_features)
X_test_numeric = pd.DataFrame(impute.transform(X_test_cont), columns=cont_features)

In [6]:
# Save numeric X
X_train_numeric.to_csv('data/X_train_numeric.csv', index=False)
X_val_numeric.to_csv('data/X_val_numeric.csv', index=False)
X_test_numeric.to_csv('data/X_test_numeric.csv', index=False)

In [7]:
# Create X_cat which contains only the categorical variables
features_cat = [col for col in X.columns if X[col].dtype in [np.object]]
X_train_cat = X_train.loc[:, features_cat]
X_val_cat = X_val.loc[:, features_cat]
X_test_cat = X_test.loc[:, features_cat]

# Fill missing values with the string 'missing'
X_train_cat.fillna(value='missing', inplace=True)
X_val_cat.fillna(value='missing', inplace=True)
X_test_cat.fillna(value='missing', inplace=True)

In [8]:
# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_val_ohe = ohe.transform(X_val_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Convert these columns into a DataFrame 
cat_columns = ohe.get_feature_names(input_features=X_train_cat.columns)
X_train_cat = pd.DataFrame(X_train_ohe.todense(), columns=cat_columns)
X_val_cat = pd.DataFrame(X_val_ohe.todense(), columns=cat_columns)
X_test_cat = pd.DataFrame(X_test_ohe.todense(), columns=cat_columns)

In [9]:
# Save categorical X
X_train_cat.to_csv('data/X_train_cat.csv', index=False)
X_val_cat.to_csv('data/X_val_cat.csv', index=False)
X_test_cat.to_csv('data/X_test_cat.csv', index=False)

In [10]:
type(X_train)

pandas.core.frame.DataFrame

In [13]:
X_train.head().append(X_train.tail())

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
120,80,RL,NaN,21453,Pave,NaN,IR1,Low,AllPub,CulDSac,Sev,ClearCr,Norm,Norm,1Fam,SLvl,6,5,1969,1969,Flat,Metal,Plywood,Plywood,None,0.0,TA,TA,CBlock,TA,TA,Gd,ALQ,938,Unf,0,0,938,GasA,Ex,Y,SBrkr,988,0,0,988,1,0,1,0,1,1,TA,4,Typ,2,TA,Attchd,1969.0,Unf,2,540,TA,TA,Y,0,130,0,130,0,0,NaN,NaN,NaN,0,10,2006,WD,Normal
1410,60,RL,79.0,12420,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,666,Unf,0,278,944,GasA,Ex,Y,SBrkr,944,896,0,1840,1,0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2001.0,RFn,2,622,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal
22,20,RL,75.0,9742,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,8,5,2002,2002,Hip,CompShg,VinylSd,VinylSd,BrkFace,281.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,1777,1777,GasA,Ex,Y,SBrkr,1795,0,0,1795,0,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2002.0,RFn,2,534,TA,TA,Y,171,159,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
730,120,RL,39.0,5389,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1995,1996,Gable,CompShg,CemntBd,CmentBd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,1180,Unf,0,415,1595,GasA,Ex,Y,SBrkr,1616,0,0,1616,1,0,2,0,2,1,Gd,5,Typ,1,TA,Attchd,1995.0,RFn,2,608,TA,TA,Y,237,152,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,Normal
1442,60,FV,85.0,11003,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,2Story,10,5,2008,2008,Gable,CompShg,VinylSd,VinylSd,Stone,160.0,Ex,TA,PConc,Ex,TA,Av,GLQ,765,Unf,0,252,1017,GasA,Ex,Y,SBrkr,1026,981,0,2007,1,0,2,1,3,1,Ex,10,Typ,1,Ex,Attchd,2008.0,Fin,3,812,TA,TA,Y,168,52,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,Normal
1321,20,RL,NaN,6627,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,BrkSide,Feedr,Norm,1Fam,1Story,3,6,1949,1950,Hip,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,NaN,NaN,NaN,NaN,0,NaN,0,0,0,Floor,TA,N,SBrkr,720,0,0,720,0,0,1,0,2,1,TA,4,Typ,0,NaN,Detchd,1955.0,Unf,1,287,TA,Fa,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,7,2008,WD,Normal
112,60,RL,77.0,9965,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2007,2007,Gable,CompShg,VinylSd,VinylSd,Stone,220.0,Gd,TA,PConc,Ex,TA,Av,GLQ,984,Unf,0,280,1264,GasA,Ex,Y,SBrkr,1282,1414,0,2696,1,0,2,1,4,1,Ex,10,Typ,1,Gd,BuiltIn,2007.0,Fin,3,792,TA,TA,Y,120,184,0,0,168,0,NaN,NaN,NaN,0,10,2007,New,Partial
805,20,RL,91.0,12274,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,Somerst,Norm,Norm,1Fam,1Story,7,5,2008,2008,Gable,CompShg,VinylSd,VinylSd,Stone,256.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,1417,1417,GasA,Ex,Y,SBrkr,1428,0,0,1428,0,0,2,0,3,1,Ex,6,Typ,0,NaN,Attchd,2008.0,RFn,2,554,TA,TA,Y,0,60,0,0,0,0,NaN,NaN,NaN,0,7,2008,New,Partial
412,20,FV,NaN,4403,Pave,NaN,IR2,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,7,5,2009,2009,Gable,CompShg,MetalSd,MetalSd,Stone,432.0,Ex,TA,PConc,Ex,TA,Av,GLQ,578,Unf,0,892,1470,GasA,Ex,Y,SBrkr,1478,0,0,1478,1,0,2,1,2,1,Gd,7,Typ,1,Gd,Attchd,2009.0,Fin,2,484,TA,TA,Y,0,144,0,0,0,0,NaN,NaN,NaN,0,6,2010,New,Partial
854,20,RL,102.0,17920,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,4,1955,1974,Hip,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,CBlock,TA,TA,Mn,ALQ,306,Rec,1085,372,1763,GasA,TA,Y,SBrkr,1779,0,0,1779,1,0,1,1,3,1,TA,6,Typ,1,Gd,Attchd,1955.0,Unf,2,454,TA,TA,Y,0,418,0,0,312,0,Na

In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1051 entries, 120 to 854
Data columns (total 79 columns):
MSSubClass       1051 non-null int64
MSZoning         1051 non-null object
LotFrontage      857 non-null float64
LotArea          1051 non-null int64
Street           1051 non-null object
Alley            69 non-null object
LotShape         1051 non-null object
LandContour      1051 non-null object
Utilities        1051 non-null object
LotConfig        1051 non-null object
LandSlope        1051 non-null object
Neighborhood     1051 non-null object
Condition1       1051 non-null object
Condition2       1051 non-null object
BldgType         1051 non-null object
HouseStyle       1051 non-null object
OverallQual      1051 non-null int64
OverallCond      1051 non-null int64
YearBuilt        1051 non-null int64
YearRemodAdd     1051 non-null int64
RoofStyle        1051 non-null object
RoofMatl         1051 non-null object
Exterior1st      1051 non-null object
Exterior2nd      1051 non-

In [16]:
X_train.isna().sum().sum()

4982